# Build label dataframe

## Imports

In [1]:
import os
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa

import OAI_Utilities as utils # ln -s ../../OAI/notebooks/OAI_Utilities.py

## Constants

In [2]:
OAI_PKL_PATH = '/Users/brandong.hill/code/OAI/notebooks/pkl/'
idxSlc = pd.IndexSlice

VARIABLES_OF_INTEREST = [
    "(0008, 0070) Manufacturer",
    "(0008, 1090) Manufacturer's Model Name",
    "(0012, 0030) Clinical Trial Site ID",
    "(0018, 1000) Device Serial Number",
]

food_var = 'FFQ18'

## Read in data

In [3]:
enrollees_df = pd.read_pickle(open(os.path.join(OAI_PKL_PATH, 'enrollees_values.pkl'), 'rb' ))  # RACE
allclinical_df = pd.read_pickle(open(os.path.join(OAI_PKL_PATH, 'allclinical_values.pkl'), 'rb' )) # food_var
xr_df = pd.read_pickle(open(os.path.join(OAI_PKL_PATH, 'xray_values.pkl'), 'rb' ))
metadata_df = pd.read_pickle(open('../../OAI_DICOM/pkl/dicom_metadata_df.pkl', 'rb'))

In [4]:
# What do OAI records say about what X-rays we have?

# Only PA view
xr_df = xr_df[xr_df['EXAMTP'] == 'Bilateral PA Fixed Flexion Knee'].copy(deep=True)
print('Total entries: {:,}'.format(len(xr_df)))
print('Entries with no barcode: {:,}'.format(xr_df['XRBARCD'].value_counts(dropna=False).at['']))

# Drop those without a barcode. No barcode == no file
xr_df = xr_df[xr_df['XRBARCD'] != ''].copy(deep=True)

# Truncate the '0166' that starts each file name
xr_df['XRBARCD'] = xr_df['XRBARCD'].str[4:]
print('Unique barcodes: {:,}'.format(len(xr_df['XRBARCD'].unique())))

Total entries: 31,767
Entries with no barcode: 5,245
Unique barcodes: 26,522


In [5]:
# What about the records from actual unarchived files?
ids_with_files = set(pd.read_pickle(open(os.path.join('../../OAI_DICOM/pkl/dicom_metadata_df.pkl'), 'rb' )).index)
print(len(ids_with_files))

26520


In [6]:
# Which files are missing? In records but not the archive
missing = set(xr_df['XRBARCD']) - ids_with_files
missing

{'02048905', '02258101'}

In [7]:
# What do we know about the two missing files?
xr_df[xr_df['XRBARCD'].isin(list(missing))]

,,VERSION,ACCEPT,ALIGN,CENTER,DEPICT,EXAMTP,EXPOSE,MOTION,POSITN,XNDREAS,XRBARCD,XRCOMP,XRDATE,XRSIDE,XRTECID
ID,Visit,,,,,,,,,,,,,,,
9417544,V01,1.2.2,'Y': QCd and found to be acceptable,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,Bilateral PA Fixed Flexion Knee,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,.A: Not Expected,02048905,1: Yes,2007-04-23,3: Bilateral,A083
9354236,V03,3.2.2,'Y': QCd and found to be acceptable,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,Bilateral PA Fixed Flexion Knee,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,.A: Not Expected,02258101,1: Yes,2007-08-07,3: Bilateral,A049


Actually QCd, found to be ok, but not included. Probably a mistake.

In [8]:
# Drop the missing files
xr_df = xr_df[~xr_df['XRBARCD'].isin(list(missing))]
print(len(xr_df))

26520


# QC

In [9]:
# What QC checks were failed?
cols = ['ALIGN', 'CENTER', 'DEPICT', 'EXPOSE', 'MOTION', 'POSITN']
for col in cols:
    tmp_df = xr_df[xr_df[col] == "'X': Does not meet QC standard"]
    print(col, '\n',  len(tmp_df))

ALIGN 
 1594
CENTER 
 322
DEPICT 
 6
EXPOSE 
 0
MOTION 
 9
POSITN 
 199


In [10]:
# How many were completely rejected?
utils.value_counts(xr_df['ACCEPT'])

'YD': Not QCd and accepted by default          15976
'Y': QCd and found to be acceptable            10338
'NR': QCd unacceptable, chosen for release       200
NaN                                                4
'NA': QCd unacceptable, no better available        2
Name: ACCEPT, dtype: int64

In [11]:
# What's the deal with NaNs?
xr_df[xr_df['ACCEPT'].isna()]

,,VERSION,ACCEPT,ALIGN,CENTER,DEPICT,EXAMTP,EXPOSE,MOTION,POSITN,XNDREAS,XRBARCD,XRCOMP,XRDATE,XRSIDE,XRTECID
ID,Visit,,,,,,,,,,,,,,,
9414263,V10,10.2.2,NaN,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,Bilateral PA Fixed Flexion Knee,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,.A: Not Expected,03801401,1: Yes,2013-02-08,3: Bilateral,3087
9465131,V10,10.2.2,NaN,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,Bilateral PA Fixed Flexion Knee,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,.A: Not Expected,03996001,1: Yes,2013-02-01,3: Bilateral,3087
9639088,V10,10.2.2,NaN,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,Bilateral PA Fixed Flexion Knee,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,.A: Not Expected,03958801,1: Yes,2013-02-13,3: Bilateral,3129
9649566,V10,10.2.2,NaN,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,Bilateral PA Fixed Flexion Knee,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,.A: Not Expected,03800801,1: Yes,2013-02-06,3: Bilateral,3080


In [12]:
# What about the NAs?
xr_df[xr_df['ACCEPT'] == "'NA': QCd unacceptable, no better available"]

,,VERSION,ACCEPT,ALIGN,CENTER,DEPICT,EXAMTP,EXPOSE,MOTION,POSITN,XNDREAS,XRBARCD,XRCOMP,XRDATE,XRSIDE,XRTECID
ID,Visit,,,,,,,,,,,,,,,
9803586,V01,1.2.2,"'NA': QCd unacceptable, no better available",'X': Does not meet QC standard,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,Bilateral PA Fixed Flexion Knee,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,.A: Not Expected,01631304,1: Yes,2006-04-21,3: Bilateral,E008
9932578,V01,1.2.2,"'NA': QCd unacceptable, no better available",' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,Bilateral PA Fixed Flexion Knee,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,' ' : Not QCd or meets QC standard,.A: Not Expected,01644104,1: Yes,2006-04-20,3: Bilateral,E008


# Create the final dataframe

In [13]:
no_answer = ['.E: Non-Exposed Control', '.R: Refused', '.A: Not Expected', '.M: Missing']

# Drop rows without useable info
allclinical_df = allclinical_df[~allclinical_df[food_var].isna()]
allclinical_df = allclinical_df[~allclinical_df[food_var].isin(no_answer)]
print('{:,}'.format(len(allclinical_df)))

no_answer = ['.R: Refused', '.D: Don t Know/Unknown/Uncertain']
enrollees_df = enrollees_df[~enrollees_df['RACE'].isna()]
enrollees_df = enrollees_df[~enrollees_df['RACE'].isin(no_answer)]
print('{:,}'.format(len(enrollees_df)))

cohort = set(allclinical_df.index.get_level_values('ID')) & set(enrollees_df.index)
print('{:,}'.format(len(cohort)))

enrollees_df = enrollees_df.loc[list(cohort)]
allclinical_df = allclinical_df.loc[list(cohort)]

4,654
4,791
4,650


In [14]:
barcode_site_id_df = pd.DataFrame(xr_df['XRBARCD'].reset_index('Visit'))
print('{:,}'.format(len(barcode_site_id_df)))

# Result= XRBARCD: ID, Visit, SITE, RACE, food_var, '(0008, 1090) Manufacturer's Model Name', '(0012, 0030) Clinical Trial Site ID', ....
barcode_site_id_df = barcode_site_id_df.join(allclinical_df[food_var].reset_index('Visit', drop=True), how='inner')
barcode_site_id_df = barcode_site_id_df.join(enrollees_df[['SITE', 'RACE']], how='inner')  # Add hospital site and patient race
barcode_site_id_df = barcode_site_id_df.reset_index('ID').set_index('XRBARCD')  # Switch to index by barcode
barcode_site_id_df = barcode_site_id_df.join(metadata_df[VARIABLES_OF_INTEREST]) # Add Mfg model, and Clinical Site ID (xray machine location)
print('{:,}'.format(len(barcode_site_id_df)))  # Sanity check, the joins shouldn't be increase the number of entries
print('{:,}'.format(len(barcode_site_id_df['ID'].unique())))

26,520
25,743
4,648


In [15]:
# Simplify column names
barcode_site_id_df = barcode_site_id_df.rename({'(0008, 0070) Manufacturer': 'MFG',
                           "(0008, 1090) Manufacturer's Model Name": 'MODEL', 
                           '(0012, 0030) Clinical Trial Site ID': 'XRAY SITE',
                           '(0018, 1000) Device Serial Number': 'SERIAL'}, axis=1)

In [16]:
# Simplify class values
for col in barcode_site_id_df.select_dtypes(include=['category']).columns:
    barcode_site_id_df[col] = barcode_site_id_df[col].cat.remove_unused_categories()
    
barcode_site_id_df[food_var] = barcode_site_id_df[food_var].cat.rename_categories({
    '1: Never' : 1,
    '2: A few times per year' : 2,
    '3: Once per month' : 3,
    '4: 2-3 times per month' : 4,
    '5: Once per week' : 5,
    '6: Twice per week' : 6,
    '7: 3-4 times per week' : 7,
    '8: 5-6 times per week' : 8,
    '9: Every day':9 })

barcode_site_id_df['RACE'] = barcode_site_id_df['RACE'].cat.rename_categories({
    '0: Other Non-white' : 'O',
    '1: White or Caucasian' : 'W',
    '2: Black or African American' : 'B',
    '3: Asian' : 'A' })

In [23]:
barcode_site_id_df

,ID,Visit,FFQ18,SITE,RACE,MFG,MODEL,XRAY SITE,SERIAL
XRBARCD,,,,,,,,,
00839603,9000099,V00,1,B,W,NaN,NaN,10,NaN
01653203,9000099,V01,1,B,W,LS100,Lumisys,10,NaN
02249203,9000099,V03,1,B,W,NaN,NaN,10,NaN
02722403,9000099,V05,1,B,W,NaN,NaN,10,NaN
03279101,9000099,V06,1,B,W,FUJIFILM Corporation,NaN,10,NaN
...,...,...,...,...,...,...,...,...,...
01317504,9999878,V01,3,C,W,Agfa-Gevaert AG,ADC_5146,46,2205
02438501,9999878,V05,3,C,W,"""GE Healthcare""","""Definium 5000""",46,NaN
03071301,9999878,V06,3,C,W,"""GE Healthcare""","""Definium 5000""",46,NaN


# Write to Parquet

In [18]:
table = pa.Table.from_pandas(barcode_site_id_df)
pq.write_table(table, 'xray_shortcutting_labels.parquet')

In [19]:
df_test_read = pd.read_parquet('xray_shortcutting_labels.parquet')